In [1]:
import json
import os
import pandas as pd
import numpy as np
import glob

In [2]:
df_all = pd.DataFrame()
path = "Json_file/"
all_files = glob.glob(os.path.join(path, "*.json"))
all_files.sort()

In [3]:
#df_activities = pd.read_csv('activities.csv')
df_timezone = pd.read_csv('timezone_dict.csv')
df_actorid = pd.read_csv('actorid_dict.csv')

dict_actorid = dict(zip(df_actorid.actorId, df_actorid.key))
dict_timezone = dict(zip(df_timezone['timezone'], df_actorid['key']))
#dict_cms = dict(zip(df_activities.id, df_activities['CMS Type']))

In [43]:
def read_json_file(json_name):
    df = []
    try:
        for line in open(json_name, 'r'):
            try:
                df.append(json.loads(line))
            except:
                pass
    except:
        pass
    df = pd.DataFrame(df)
    return df

In [5]:
def manual_label(i):
    if i == 'https://coles.hub.clearlrs.com':
        return "home"
    
    if "activities" in i:
        return "activities"
        
    if "course_id" in i:
        return 'course_id'
    
    if "notifications" in i:
        return 'notifications'
    
    if "course/" in i:
        return "learningseat_course"
    
    if "colesplay" in i:
        return "colesplay"
    
    if 'https://coles.hub.clearlrs.com' not in i:
        return 'others'

    i = i.split('/')[-1] 
    return i

In [6]:
def clean_first(df,name):
    df = df.drop(["id"],axis = 1)
    df = df.drop(["response"],axis = 1)
    df = df.drop(["rating"],axis = 1)
    df = df.drop(["other"],axis = 1)
    
    
    # ActorID
    list_actorid = df['actorId'].tolist()
    temp_list = []
    for i in list_actorid:
        try:
            temp_list.append(dict_actorid[i])
        except KeyError:
            temp_list.append('nan')
            
    df['actorId'] = temp_list
    print("actorId")
    
    
    # Verb: keep the last word
    list_verb = df['verb'].tolist()
    list_verb = [ i.split('/')[-1] for i in list_verb]
    df['verb'] = list_verb
    print("verb")
    
    
    # Type: keep last word
    list_type = df['type'].tolist()
    list_type = [ str(i).split('/')[-1] for i in list_type]
    for i in range(0,len(list_type)):
        if list_type[i] == 'nan':
            list_type[i] = pd.NA
    df['type'] = list_type 
    print("type")
    
    # Is manager: YN to 10
    df.loc[(df["Is Manager"] == "Y"), "Is Manager"] = 1
    df.loc[(df["Is Manager"] == "N"), "Is Manager"] = 0
    print("manager")
    
    
    # Is new starter: YN to 10
    df.loc[(df["Is New Starter"] == "Y"), "Is New Starter"] = 1
    df.loc[(df["Is New Starter"] == "N"), "Is New Starter"] = 0
    print("Starter")
    
    
    # Activities
    list_activity = np.array(df['activity'])
    temp_list = []
    for i in list_activity:
        res = manual_label(i)
        temp_list.append(res)

    df.insert(4, 'activity_url', df['activity'].to_list())
    df['activity'] = temp_list
    print('activity')
    
    
    # ancestors
    list_ancestors = np.array(df['ancestors'])
    temp_list = []
    for i in list_ancestors:
        lines = str(i)[1:-1].split(",")
        temp_list.append([manual_label(lines[i][14:-1]) for i in range(len(lines)) if i % 2 == 0])
    df['ancestors'] = temp_list
    print("ancestors")
    
    
    # Contained
    list_contain = np.array(df.contained)
    temp_list = []
    for i in range(0,len(list_contain)):
        if str(list_contain[i]) == '<NA>' or str(list_contain[i]) == 'nan':
            temp_list.append(pd.NA)
        else:
            target_id = str(list_contain[i]).split(',')[0][13:-1]
            result = []
            try:
                result.append(dict_actorid[target_id])
            except KeyError:
                result.append('unrecorded')

            contained_verb = str(list_contain[i]).split(',')[1][10:-2].split('/')[-1]
            result.append(contained_verb)
            temp_list.append(result)
    df.contained = temp_list
    print("contained")
    
    
    # Timezone
    list_timezone = df['Time Zone'].unique()
    temp_list = [dict_timezone[i] for i in df['Time Zone']]
    df['Time Zone'] = temp_list
    print("time zone")
    
    df.to_csv(name, index = False)
    print('Finish')

In [44]:
#0
file_list = [0,30,60,90,120,150,180,210,240,270,300,330,365]
for i in range(len(file_list)-1):
    print(file_list[i],file_list[i+1])
    a = file_list[i]
    b = file_list[i+1]
    
    df_all = pd.DataFrame()
    count = 1
    for j in all_files[int(a):int(b)]:
        df_all = df_all.append(read_json_file(j))
        print(j + "  " + str(count))
        count+=1
    
    name = '12_CSV/' + str(i+10) + '.csv'
    print(name)
    clean_first(df_all,name)
    print('')

300 330
Json_file/2022-04-27.json  1
Json_file/2022-04-28.json  2
Json_file/2022-04-29.json  3
Json_file/2022-04-30.json  4
Json_file/2022-05-01.json  5
Json_file/2022-05-02.json  6
Json_file/2022-05-03.json  7
Json_file/2022-05-04.json  8
Json_file/2022-05-05.json  9
Json_file/2022-05-06.json  10
Json_file/2022-05-07.json  11
Json_file/2022-05-08.json  12
Json_file/2022-05-09.json  13
Json_file/2022-05-10.json  14
Json_file/2022-05-11.json  15
Json_file/2022-05-12.json  16
Json_file/2022-05-13.json  17
Json_file/2022-05-14.json  18
Json_file/2022-05-15.json  19
Json_file/2022-05-16.json  20
Json_file/2022-05-17.json  21
Json_file/2022-05-18.json  22
Json_file/2022-05-19.json  23
Json_file/2022-05-20.json  24
Json_file/2022-05-21.json  25
Json_file/2022-05-22.json  26
Json_file/2022-05-23.json  27
Json_file/2022-05-24.json  28
Json_file/2022-05-25.json  29
Json_file/2022-05-26.json  30
12_CSV/10.csv
actorId
verb
type
manager
Starter
activity
ancestors
contained
time zone
Finish

330 36